## NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

print(nlp.pipe_names)

In [ ]:
from app.templates.meat_sale.domain_model_template import meat_sale_domain_model_template as dm
from app.templates.meat_sale.contract_spec_template import meat_sale_contract_spec_template as cs
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

ob = cs.obligations['payment']
ob.to_sym()

In [ ]:
from app.templates.meat_sale.contract_template import get_template

contract_template = get_template()

In [ ]:
# <amt> (“amount”) in <curr> (“currency”)
# 100 USD
# 99.30 CAD 
test_suite = [
    ['$10.50 in USD', 'USD'], 
    ['$2000 USD', 'USD'], 
    ['500 CAD', 'CAD'], 
    ['$500.00 CAD', 'CAD'], 
    ['USD 787', 'USD'], 
    #...
]

index = 4
sentence = test_suite[index][0]

doc = nlp(sentence)

summarizer.summarize(sentence)

In [ ]:
from app.src.rules.meat_sale.payment_amount.score_based_extractor import ScoreBasedExtractor
from app.src.rules.meat_sale.delivery_location.basic_scorer import BasicScorer
from app.src.rules.meat_sale.payment_amount.currency_scorer import CurrencyScorer
from app.src.rules.meat_sale.payment_amount.matchers import get_validation_matcher
from app.src.rules.shared.match_validator import MatchValidator

key = 'PAYMENT_AMOUNT'

matcher = get_validation_matcher(nlp)

validator = MatchValidator(matcher)

allowed_currs = ['USD', 'CAD'] # Get this from the domain model
currency_scorer = CurrencyScorer(allowed_currs)

currency_scorers = [
    BasicScorer('X', 0),
    currency_scorer
]

currency_extractor = ScoreBasedExtractor(nlp, validator, currency_scorers)



In [ ]:
from app.classes.contract_update_request import ContractUpdateRequest

req = ContractUpdateRequest(
    contract_template,
    key,
    sentence,
    doc
)

currency = currency_extractor.extract(req)

In [ ]:
print(currency)

In [ ]:
from app.src.rules.meat_sale.payment_amount.amount_scorer import AmountScorer

amount_scorer = AmountScorer(nlp)

amount_scorers = [
    BasicScorer('0', 0),
    amount_scorer
]

amount_extractor = ScoreBasedExtractor(nlp, validator, amount_scorers)

In [ ]:
req = ContractUpdateRequest(
    contract_template,
    key,
    sentence,
    doc
)

amount = amount_extractor.extract(req)

In [ ]:
print(amount)